# Análisis de datos exploratorio

- **Ejecutar primero data.ipynb (linux)**

## Cargar datos

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_json("features.json", orient='records', lines=True)

In [ ]:
df.head(5)

In [ ]:
df.info()

## Creacion del Word Embedding (Glove 100d)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# max_words = 1000
max_sequence_length = 250
embedding_dim = 100

In [ ]:
# Tokenizar la data (BOW)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['features'])
word_index = tokenizer.word_index
vocab_size = len(word_index)
vocab_size

In [ ]:
# Realizar el padding
sequences = tokenizer.texts_to_sequences(df['features'])
padded_seq = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [ ]:
# Crear el embedding diccionario de nuestro dataset
embedding_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

In [ ]:
# Crear el embedding matrix de nuestro dataset
embedding_matrix = np.zeros((vocab_size+1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix[1]

In [ ]:
padded_seq[0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_seq, df['label'], test_size=0.30, random_state=42, stratify=df['label'])

In [ ]:
tokenizer_json = tokenizer.index_word
inv_map = {v: k for k, v in tokenizer_json.items()}

In [ ]:
import json

# Nombre del archivo de salida
nombre_archivo = 'vocab.json'

# Guarda el vocabulario invertido en un archivo JSON
with open(nombre_archivo, 'w') as archivo:
    json.dump(inv_map, archivo)

print(f'Vocabulario invertido guardado en {nombre_archivo}')


# Modelating

## Redes Neuronales

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding, LSTM

In [ ]:
# Construir y entrenar la red neuronal
model = Sequential()

# Capa de Embedding
model.add(
    Embedding(
        input_dim=vocab_size + 1,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        input_length=max_sequence_length,
        trainable=False,
    )
)
model.add(Dropout(0.2))
# model.add(LSTM(units=128, recurrent_dropout=0.25, dropout=0.25))
model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
history = model.fit(
    np.array(X_train),
    y_train,
    epochs=5,
    batch_size=32,
    verbose=True,
    validation_data=(np.array(X_test), y_test),
    workers=4,
)

In [ ]:
model.save("modelo.h5")

# Ver importancia de palabras

In [ ]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import string

In [ ]:
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text
lemmatizer = WordNetLemmatizer()

In [ ]:
def process_text(text):
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    text = re.sub(r'\$\w*', '', text)
    # remove old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '', text)
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    # remove hashtags
    text = re.sub(r'#', '', text)
    text = str(re.sub("\S*\d\S*", "", text).strip())
    text = decontract(text)

    # tokenize texts
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tokens = tokenizer.tokenize(text)

    texts_clean = []
    for word in tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation+'...'):  # remove punctuation
            #
            stem_word = lemmatizer.lemmatize(word,"v")  # Lemmatizing word
            texts_clean.append(stem_word)

    return " ".join(texts_clean)

In [ ]:
%pip install lime

In [ ]:
from lime.lime_text import LimeTextExplainer
class_names=['NotFake','IsFake']
explainer= LimeTextExplainer(class_names=class_names)

def predict_proba(arr):
  processed=[]
  for i in arr:
    processed.append(process_text(i))
  list_tokenized_ex = tokenizer.texts_to_sequences(processed)
  Ex = pad_sequences(list_tokenized_ex, maxlen=max_sequence_length)
  pred=model.predict(Ex)
  returnable=[]
  for i in pred:
    temp=i[0]
    returnable.append(np.array([1-temp,temp])) #I would recommend rounding temp and 1-temp off to 2 places
  return np.array(returnable)

In [ ]:
print("Actual rating",df['label'][2])
explainer.explain_instance(df['text'][2],predict_proba).show_in_notebook(text=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Bolsa de palabras
vectorizer = CountVectorizer(max_features=5000, stop_words=stopwords.words('english'))

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(df['features'][0])

palabras = vectorizer.get_feature_names_out()

frecuencias = texto_features.toarray()

In [ ]:
frecuencias_totales = frecuencias.sum(axis=0)

# Obtener las palabras más frecuentes y sus frecuencias
palabras_mas_frecuentes = [palabras[i] for i in frecuencias_totales.argsort()[::-1][:10]]
frecuencias_mas_frecuentes = [frecuencias_totales[i] for i in frecuencias_totales.argsort()[::-1][:10]]

# Crear un gráfico de barras horizontal
plt.figure(figsize=(10, 6))
plt.barh(palabras_mas_frecuentes, frecuencias_mas_frecuentes, color='skyblue')
plt.xlabel('Frecuencia')
plt.ylabel('Palabra')
plt.title('Palabras más frecuentes')
plt.gca().invert_yaxis()  # Invertir el eje y para mostrar las palabras más frecuentes arriba
plt.show()

## Evaluation

In [ ]:
# Evaluar el modelo
loss, accuracy = model.evaluate(np.array(X_test), y_test)
print(f'Accuracy: {accuracy}')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
predictions = model.predict(np.array(X_test))
predictions = list(map(lambda x: 1 if (x > 0.5) else 0, predictions))

cm = confusion_matrix(y_test, predictions,labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
disp.plot()

In [ ]:
print(classification_report(y_test, list(predictions), digits=4))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()